# Twitter Crawler 

## Import Libraries

In [ ]:
import tweepy
import csv
from tweepy import OAuthHandler

## Determine output file name and twitter access code

In [ ]:
output_file = './data/chinatradewar_tweets.csv'

# YK's Code
consumer_key    = 'xjYZsoUMQC5NuLpHENDAR0TVm'
consumer_secret = 'W6l7yeS7AI8ZfQSEtkBBIID9Uv2zCgX2q5TPoZykiUKJa5QdFn'
access_token    = '1168774617281912832-MY3EbsImXQbrkfaOxb1VEJ5tWWn4BK'
access_secret   = 'vzh3KPViXm0201CbNjVaG0hOJV7hKv3FSCspzCEtJhNwa' 

# Darren's Code
# consumer_key    = 'g6KDuWXcSWxDmkZIdlQVNanrR'
# consumer_secret = '4U5ceThn0ihOTUcUXjyhclab86kI3h9vWXWCCYDtOXUZqPqXJY'
# access_token    = '1112632162413510656-6F2FGTig1pN0R737tSPMEkzPvzZ2F4'
# access_secret   = 'SqRM99877qyQAUO2ai453nWBVJb8A6kSzEdCpedzkuuur'

## Tweepy

### Set Authentication

In [ ]:
# Consumer/Access key/secret/token obtained from Twitter
# You should have created a Twitter app and gotten these keys.
# Do NOT share your key/secret/token with other students.

# The following two lines create an authorization object with your above authentication info.
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# This line finally calls Twitter's Rest API.
api = tweepy.API(auth)

## Crawling using search term

In [ ]:
search_terms = ['chinatradewar']

# crawled_list = []
tweet_list = []

# results = api.search(q=query, lang=language)
# This FOR loop will retrieve the latest 10 tweets - based on search term(s)
for tweet in tweepy.Cursor(api.search, q=search_terms, lang='en').items(10000):
    user_dict = {}
    tweet_dict = {}
    # print out user's screen name & tweet text
    print("----------------------------------------------------")
    print ('Tweet by: @' + tweet.user.screen_name)
    print("====================================================")
    print('       ' + tweet.text)
    tweet_dict['username'] = tweet.user.screen_name
    tweet_dict['tweet'] = tweet.text
#     user_dict[tweet.user.screen_name] = tweet.text 
    tweet_list.append(tweet_dict)
#     crawled_list.append(user_dict)

### Output to CSV

In [ ]:
keys = tweet_list[0].keys()
with open(output_file, 'w', encoding='utf-8-sig') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(tweet_list)

## Twython

In [ ]:
import json

# Consumer/Access key/secret/token obtained from Twitter
# You should have created a Twitter app and gotten these keys.
# Do NOT share your key/secret/token with other students.

credentials = {}
credentials['CONSUMER_KEY'] = consumer_key
credentials['CONSUMER_SECRET'] = consumer_secret
credentials['ACCESS_TOKEN'] = access_token
credentials['ACCESS_SECRET'] = access_secret

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [ ]:
'''
In the next step, we'll import the Twython class, instantiate an object of it, 
and create our search query.

We'll use only four arguments in the query:
   q, result_type, count and lang,
      respectively
   for the search keyword, type, count, and language
      of search results.

Twitter also defines other arguments to fine-tune the search query, which can be found here:
https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets.html
'''

# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': 'chinatradewar',
        'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        }

In [ ]:
'''
Finally, we can use our Twython object to call the search method, 
which returns a dictionary of search_metadata and statuses - the queried results.

We'll only look at the statuses part, and save a portion of all information 
in a pandas dataframe, to present it in a table.
'''

# Check if your Anaconda Python 3.7 has "pandas" package
import pandas as pd

# Search tweets
dict_ = {
        'user': [],
        'date': [],
        'text': [],
        'favorite_count': []
        }

for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(5)

### Output to CSV

In [ ]:
# article_df = article_df[article_df['content'] != '']
df.reset_index(drop=True)
df.to_csv(output_file, encoding='utf-8-sig')